In [71]:
import pandas as pd
import numpy as np
import scipy as sp

from nltk.corpus import stopwords
import nltk

from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

import re

from tqdm import tqdm
tqdm.pandas()

from rouge_score import rouge_scorer
from bert_score import score
from nltk.translate import bleu_score

import statistics

In [2]:
SRC_DATA = '../../data/data_clean/polisumm_final.csv'

In [3]:
data = pd.read_csv(SRC_DATA)

In [6]:
url_regex = r'https?:\/\/\S*'
punct_regex = '\s*[\.\?\!\|\n\:]\s*[\.\?\!\|\n\:]*\s*'
sw = stopwords.words('english')

# Functions 

In [7]:
def process_texts(cat_text, min_toks = 3):
    new_text = re.sub(url_regex, '', cat_text)
    new_text = ' '.join([term for term in new_text.split() if term not in sw])
    text_l   = re.split(punct_regex, new_text)
    text_l   = [sent for sent in text_l if len(sent.split()) > min_toks]
    
    return text_l

def filter_min_toks(cat_text, min_toks = 3):
    new_text = re.sub(url_regex, '', cat_text)
    text_l    = re.split(punct_regex, new_text)
    
    new_text = ' '.join([term for term in new_text.split() if term not in sw])
    ptext_l   = re.split(punct_regex, new_text)
    text_l   = [text_l[i] for i, sent in enumerate(ptext_l) if len(sent.split()) > min_toks]
    
    return text_l

# Data Processing 

In [8]:
data['text_proc'] = data['all_texts'].progress_apply(process_texts)
data['filt_texts'] = data['all_texts'].progress_apply(filter_min_toks)

100%|██████████████████████████████████████████████████████████████████████████████| 1199/1199 [00:44<00:00, 27.19it/s]


In [ ]:
vect = TfidfVectorizer(min_df = 3, max_df = 0.8)
vect = vect.fit(data['text_proc'].fillna(''))

# Model Definition

In [ ]:
class EGCOS():
    
    def __init__(self, vectorizer):
        self.PUNCT_REGEX = '\s*[\.\?\!\|\n\:]\s*[\.\?\!\|\n\:]*\s*'

        self.vectorizer = vectorizer
        
        
    def predict(self, text, src_text = None):
        text_l = re.split(self.PUNCT_REGEX, text) if isinstance(text, str) else text
        if src_text:
            src_text_l = re.split(self.PUNCT_REGEX, src_text) if isinstance(src_text, str) else src_text
        
        text_bows = self.vectorizer.transform(text_l)
        text_bows = normalize(text_bows, norm = 'l1', axis = 1)
                
        split_preds = self.get_factorizer(text_bows)
        classes = split_preds.argmax(axis = -1).astype(bool)
        
        half1 = text_bows[classes]
        half2 = text_bows[~classes]
        
        pair_dists = pairwise_distances(half1, half2, metric = 'cosine')
        h1_idx, h2_idx = np.unravel_index(pair_dists.argmax(), pair_dists.shape)
        
        if src_text:
            pred_h1, pred_h2 = np.array(src_text_l)[classes][h1_idx], np.array(src_text_l)[~classes][h2_idx]
        else:
            pred_h1, pred_h2 = np.array(text_l)[classes][h1_idx], np.array(text_l)[~classes][h2_idx]
            
        return pred_h1, pred_h2

        
    def combine_topic_probs(self, probs1, probs2):
        combos = itertools.product(probs1, probs2)
        combos = [a + b for (a, b) in combos]
        combos = np.array(combos).reshape(len(probs1), len(probs2))
        
        return combos
        
        
    def get_factorizer(self, text_bows):
        
        factorizer = NMF(n_components = 2, init = 'nndsvd')
        return factorizer.fit_transform(text_bows)

# Evaluation 

In [ ]:
egcos = EGCOS(vect)

In [ ]:
def make_prediction(model, row):
    texts = row['text_proc']
    
    try:
        summ_1, summ_2 = model.predict(texts)
    except Exception as e:
        print(f'Error: {e}')
        return ' | '
    return summ_1 + ' | ' + summ_2

In [110]:
predictions = data.progress_apply(lambda row: make_prediction(egcos, row), axis = 1)

  9%|██████▋                                                                        | 102/1199 [00:31<05:15,  3.47it/s]

Error: init = 'nndsvd' can only be used when n_components <= min(n_samples, n_features)


 10%|████████                                                                       | 123/1199 [00:38<05:49,  3.08it/s]C:\Users\ndeas\miniconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 22%|█████████████████▌                                                             | 267/1199 [01:22<04:34,  3.39it/s]C:\Users\ndeas\miniconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
 43%|██████████████████████████████████                                             | 517/1199 [02:40<02:56,  3.87it/s]C:\Users\ndeas\miniconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve conver

In [112]:
predictions[:3]

0    economy amid coronavirus outbreak | Family 4 c...
1    What Trump's Trillion-Dollar Bailout Gets Righ...
2    I tend think thinly veiled excuse | 5 major Fa...
dtype: object

In [132]:
predictions.to_csv('preds_egcos.csv')

In [95]:
predictions = pd.read_csv('preds_egcos.csv')

In [97]:
predictions['pred1'] = predictions['0'].str.split('|').apply(lambda l: l[0])
predictions['pred2'] = predictions['0'].str.split('|').apply(lambda l: l[1])

In [96]:
refs = data['left_sum'] + ' | ' + data['right_sum']

In [98]:
eval_split = pd.concat((predictions[['pred1', 'pred2', '0']], data[['left_sum', 'right_sum']]), axis = 1)

In [99]:
eval_split['ref_comb'] = eval_split['left_sum'] + '|' + eval_split['right_sum']
eval_split['ref_comb_rev'] = eval_split['right_sum'] + '|' + eval_split['left_sum']

In [100]:
eval_split = eval_split.dropna()

### Bert Score 

In [37]:
bert_scores = []
for i, (pred, ref) in enumerate(zip(eval_set['preds'].astype(str).values, eval_set['refs'].astype(str).values)):

    bert_score = score([pred], [ref], 
                       lang = 'en',
                      use_fast_tokenizer = True,
                      batch_size = 1,
                       model_type = ''
                      )
    
    bert_scores.append(bert_score)
    print(f'Completed {i + 1} of {len(eval_set)}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Completed 1 of 1199


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Completed 2 of 1199


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Completed 1197 of 1199


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Completed 1198 of 1199


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Completed 1199 of 1199


In [87]:
bert_scores_proc = {'prec': [], 'rec': [], 'f1': []}
for bscore in bert_scores:
    bert_scores_proc['prec'].append(bscore[0].item())
    bert_scores_proc['rec'].append(bscore[1].item())
    bert_scores_proc['f1'].append(bscore[2].item())

In [88]:
with open('bert_scores.txt', 'w') as f:
    f.write(str(bert_scores_proc))

In [89]:
bprec = sum(bert_scores_proc['prec'])/len(bert_scores_proc['prec'])
brec  = sum(bert_scores_proc['rec']) / len(bert_scores_proc['rec'])
bf1   = sum(bert_scores_proc['f1']) / len(bert_scores_proc['f1'])

In [142]:
bf1

0.40854309580572995

In [112]:
eval_split

,pred1,pred2,0,left_sum,right_sum,ref_comb,ref_comb_rev
0,economy amid coronavirus outbreak,Family 4 could get $3K virus relief plan (AP)...,economy amid coronavirus outbreak | Family 4 c...,The left supports cash payments and argues tha...,The right is generally supportive of helping c...,The left supports cash payments and argues tha...,The right is generally supportive of helping c...
1,What Trump's Trillion-Dollar Bailout Gets Righ...,🎉 news every day,What Trump's Trillion-Dollar Bailout Gets Righ...,The left is supportive of both policies.,The right is critical of both policies.,The left is supportive of both policies.|The r...,The right is critical of both policies.|The le...
2,I tend think thinly veiled excuse,5 major Facebook stories broke last days,I tend think thinly veiled excuse | 5 major Fa...,"The left is skeptical of Zuckerberg’s motives,...",The right is disturbed by the free speech impl...,"The left is skeptical of Zuckerberg’s motives,...",The right is disturbed by the free speech impl...
3,*wildly* uninformed guess #USElections2020,Biden maintains 90% chance winning,*wildly* uninformed guess #USElections2020 | B...,The left is optimistic about Biden’s chances.,The right is cautiously optimistic about Trump...,The left is optimistic about Biden’s chances.|...,The right is cautiously optimistic about Trump...
4,"Biden 90% chance winning, according forecast",But doesn’t mean isn’t still path Trump,"Biden 90% chance winning, according forecast |...",The left is optimistic about Biden’s chances.,The right is cautiously optimistic about Trump...,The left is optimistic about Biden’s chances.|...,The right is cautiously optimistic about Trump...
...,...,...,...,...,...,...,...
1186,Jurors hearing testimony #JohnnyDepp's libel l...,Depp Heard definetly deserve other,Jurors hearing testimony #JohnnyDepp's libel l...,The left worries that the case will be used to...,"The right generally sympathizes with Depp, and...",The left worries that the case will be used to...,"The right generally sympathizes with Depp, and..."
1189,The best thing United States broader internati...,@JordynTrump1 @EAShepardson @Phil_Blackmon No...,The best thing United States broader internati...,The left is generally supportive of the decisi...,The right is generally supportive of the decis...,The left is generally supportive of the decisi...,The right is generally supportive of the decis...
1190,They’re going add preface explain historical c...,""" Es algo temporal",They’re going add preface explain historical c...,The left generally supports the editor’s firin...,The right opposes the editor’s firing and the ...,The left generally supports the editor’s firin...,The right opposes the editor’s firing and the ...
1191,"Apparently, Trump bad foreign relations, accor...",Latein- u Südamerika schwindet,"Apparently, Trump bad foreign relations, accor...",The left highlights the structural impediments...,The right is critical of the Biden administrat...,The left highlights the structural impediments...,The right is critical of the Biden administrat...


### ROUGE Scores

In [101]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer = True)

In [133]:
def get_rouge_scores(df, src_col, ref_col):
    rouge_scores = []
    for i, (pred, ref) in enumerate(zip(df[src_col].astype(str).values, df[ref_col].astype(str).values)):
        rscore = scorer.score(ref, pred)
        rouge_scores.append(rscore)
        print(f'Done with {i} of {len(df)}', end = '\r')
    return rouge_scores

In [134]:
pred1_l_rouge = get_rouge_scores(eval_split, 'pred1', 'left_sum')
pred1_r_rouge = get_rouge_scores(eval_split, 'pred1', 'right_sum')
pred2_l_rouge = get_rouge_scores(eval_split, 'pred2', 'left_sum')
pred2_r_rouge = get_rouge_scores(eval_split, 'pred2', 'right_sum')
comb_rouge    = get_rouge_scores(eval_split, '0', 'ref_comb')
comb_rouge_r  = get_rouge_scores(eval_split, '0', 'ref_comb_rev')

In [135]:
all_results = [pred1_l_rouge, pred1_r_rouge, pred2_l_rouge, pred2_r_rouge, comb_rouge, comb_rouge_r]
all_results_names = ['1l', '1r', '2l', '2r', 'c', 'cr']

In [136]:
all_results = [pd.DataFrame(res) for res in all_results]
for res in all_results:
    for column in res.columns:
        res[column] = res[column].apply(lambda scores: scores[-1])

In [137]:
for name, res in zip(all_results_names, all_results):
    res.columns = [name + '_' + col for col in res.columns]

In [138]:
result_data = pd.concat(all_results, axis = 1)

In [139]:
result_data['ml_rouge1'] = result_data[['1l_rouge1', '2l_rouge1']].max(axis = 1)
result_data['ml_rouge2'] = result_data[['1l_rouge2', '2l_rouge2']].max(axis = 1)
result_data['ml_rougeL'] = result_data[['1l_rougeL', '2l_rougeL']].max(axis = 1)
result_data['mr_rouge1'] = result_data[['1r_rouge1', '2r_rouge1']].max(axis = 1)
result_data['mr_rouge2'] = result_data[['1r_rouge2', '2r_rouge2']].max(axis = 1)
result_data['mr_rougeL'] = result_data[['1r_rougeL', '2r_rougeL']].max(axis = 1)
result_data['mc_rouge1'] = result_data[['c_rouge1', 'cr_rouge1']].max(axis = 1)
result_data['mc_rouge2'] = result_data[['c_rouge2', 'cr_rouge2']].max(axis = 1)
result_data['mc_rougeL'] = result_data[['c_rougeL', 'cr_rougeL']].max(axis = 1)

In [140]:
result_data[['ml_rouge1', 'ml_rouge2', 'ml_rougeL', 'mr_rouge1', 'mr_rouge2', 'mr_rougeL', 'mc_rouge1', 'mc_rouge2', 'mc_rougeL']].mean()

ml_rouge1    0.073364
ml_rouge2    0.006047
ml_rougeL    0.067778
mr_rouge1    0.070894
mr_rouge2    0.004214
mr_rougeL    0.065441
mc_rouge1    0.063005
mc_rouge2    0.003971
mc_rougeL    0.058000
dtype: float64

In [90]:
rouge1 = {'prec': [], 'rec': [], 'f1': []}
rouge2 = {'prec': [], 'rec': [], 'f1': []}
for rscore in rouge_scores:
    rouge1['prec'].append(rscore['rouge1'].precision)
    rouge1['rec'].append(rscore['rouge1'].recall)
    rouge1['f1'].append(rscore['rouge1'].fmeasure)
    
    rouge2['prec'].append(rscore['rouge2'].precision)
    rouge2['rec'].append(rscore['rouge2'].recall)
    rouge2['f1'].append(rscore['rouge2'].fmeasure)

In [91]:
with open('rouge1_scores.txt', 'w') as f:
    f.write(str(rouge1))
with open('rouge2_scores.txt', 'w') as f:
    f.write(str(rouge2))

In [92]:
rouge1_f = {key: statistics.mean(val) for key, val in rouge1.items()}
rouge2_f = {key: statistics.mean(val) for key, val in rouge2.items()}

### Diversity (Self-Bleu) 

In [50]:
def get_self_bleu(split_preds):
    self_bleus = []
    for pred_pair in split_preds:
        bleu   = bleu_score.sentence_bleu([pred_pair[0]], pred_pair[1], weights = (1.0,))
        self_bleus.append(bleu)
    return self_bleus

In [51]:
split_predictions = [pred.split('|') for pred in predictions['0']]

In [52]:
self_bleu = get_self_bleu(split_predictions)

## All Metrics 

In [94]:
print(f'Self-Bleu:            {sum(self_bleu)/len(self_bleu):.3f}')
print(f'Bert Score   (Prec):  {bprec:.3f}')
print(f'              (Rec):  {brec:.3f}')
print(f'               (F1):  {bf1:.3f}')
print(f'Rouge1 Score (Prec):  {rouge1_f["prec"]:.3f}')
print(f'              (Rec):  {rouge1_f["rec"]:.3f}')
print(f'               (F1):  {rouge1_f["f1"]:.3f}')
print(f'Rouge2 Score (Prec):  {rouge2_f["prec"]:.3f}')
print(f'              (Rec):  {rouge2_f["rec"]:.3f}')
print(f'               (F1):  {rouge2_f["f1"]:.3f}')

Self-Bleu:            0.305
Bert Score   (Prec):  0.412
              (Rec):  0.407
               (F1):  0.409
Rouge1 Score (Prec):  0.083
              (Rec):  0.056
               (F1):  0.063
Rouge2 Score (Prec):  0.005
              (Rec):  0.004
               (F1):  0.004
